### Step 1: Set up the workspace

In [32]:
import json

#GETTING MY ID:
id = open('id_confidential.json', 'r')
mi = json.load(id)
my_id = mi["azure_id"]

from azureml.core import Workspace

ws = Workspace.create(name="bcy",
                      subscription_id = my_id,
                      resource_group = "bkrp",
                      location = "centralindia")

from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="bank_model",
                                  workspace=ws)

Deploying KeyVault with name bcykeyvaultf5bc19e53ed44.
Deploying AppInsights with name bcyinsightsc5ddd8b6c08b4.
Deployed AppInsights with name bcyinsightsc5ddd8b6c08b4. Took 8.83 seconds.
Deploying StorageAccount with name bcystorage9fcadd89a58d4b.
Deployed KeyVault with name bcykeyvaultf5bc19e53ed44. Took 25.96 seconds.
Deploying Workspace with name bcy.
Deployed StorageAccount with name bcystorage9fcadd89a58d4b. Took 31.13 seconds.
Deployed Workspace with name bcy. Took 42.51 seconds.
Registering model bank_model


In [33]:
scorepy = """import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('bank_model')
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data'][0]
        data = pd.DataFrame(data)

        result = model.predict(data)

        return json.dumps({"result": result.tolist()})
    except Exception as e:
        return json.dumps({"error": str(e)})
"""

with open("score.py", "w") as file_score:
    file_score.write(scorepy)

### Step 2: Inference configuration

In [34]:
from azureml.core.environment import Environment
virtual_env = Environment("env_bankrupt")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn'])


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )

aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4)

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\blast\AppData\Local\Temp\ipykernel_5296\778469531.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [35]:
service.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"


In [36]:
scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

In [37]:
print(scoring_uri)

http://dc93f2f1-cd7f-4172-9069-2a8f265313e7.centralindia.azurecontainer.io/score
